In [2]:
import pyproj
import geopandas as gpd

### Define Function

In [8]:
def read_shape(gdf):
    '''
    read the shape of dataframe to identify the 
    count of features (rows) and fields (columns).
    '''
    features=gdf.shape[0]
    fields=gdf.shape[1]
    return features, fields


def field_name_type(gdf):
    '''
    return the name and type of each field (column)
    '''
    return gdf.columns, gdf.dtypes


def bounding_box(gdf):
    '''
    return the bounding box information
    '''
    return gdf.total_bounds


def get_projected_coordinate_system(gdf):
    '''
    retrieve required information for defining the projected 
    coordinate system
    '''
    import re
    name = gdf.crs.name
    datum = gdf.crs.to_dict()['datum']
    units = gdf.crs.to_dict()['units']
    wkt = gdf.crs.to_wkt()
    cleaned_crs_string = re.sub(r"\[\s*([^\[\]]+)\s*\]", r"\1", wkt) 
    cleaned_crs_string = cleaned_crs_string.replace('"', "'")
    return name, datum, units, cleaned_crs_string
    
    
def get_geographic_coordinate_system(gdf):   
    '''
    retrieve required information for defining the geographical 
    coordinate system
    '''
    import sqlite3
    # Open a connection to the GeoPackage using sqlite3
    conn = sqlite3.connect('./nextgen_18.gpkg')
    # Query the metadata table for the GCS information
    query = "SELECT srs_name, srs_id FROM gpkg_spatial_ref_sys WHERE organization = 'EPSG' AND definition LIKE 'GEOGCS%'"
    gcs_data = conn.execute(query).fetchall()
    # Print the retrieved GCS data
    for row in gcs_data:
        return row     

### Read the Shapefile

In [64]:
# the example shapefile can be found at https://www.hydroshare.org/resource/fed970c19b9c41928f2591adf5b64dd1/
gdf = gpd.read_file('logan-watershed.shp')

### Retrieve Metadata

In [9]:
features, fields = read_shape(gdf)
print(features, fields)

7 10


In [33]:
colnames, dtypes = field_name_type(gdf)
print(dtypes)

objectid       float64
areaacres      float64
areasqkm       float64
states          object
huc12           object
name            object
tohuc           object
shape_Leng     float64
Shape_Area     float64
geometry      geometry
dtype: object


In [24]:
print(gdf.total_bounds)
# (min_x, min_y, max_x, max_y)

[-1229614.57918111   303353.62119147 -1200859.26639075   346695.94508645]


In [63]:
name, datum, units, cleaned_crs_string = get_projected_coordinate_system(gdf)
print(name)
print(datum)
print(units)
print(cleaned_crs_string)

North_America_Albers_Equal_Area_Conic
NAD83
m
PROJCRS['North_America_Albers_Equal_Area_Conic',BASEGEOGCRS['NAD83',DATUM['North American Datum 1983',ELLIPSOID['GRS 1980',6378137,298.257222101,LENGTHUNIT'metre',1]],PRIMEM['Greenwich',0,ANGLEUNIT'degree',0.0174532925199433],ID'EPSG',4269],CONVERSION['unnamed',METHOD['Albers Equal Area',ID'EPSG',9822],PARAMETER['Latitude of false origin',40,ANGLEUNIT'degree',0.0174532925199433,ID'EPSG',8821],PARAMETER['Longitude of false origin',-96,ANGLEUNIT'degree',0.0174532925199433,ID'EPSG',8822],PARAMETER['Latitude of 1st standard parallel',20,ANGLEUNIT'degree',0.0174532925199433,ID'EPSG',8823],PARAMETER['Latitude of 2nd standard parallel',60,ANGLEUNIT'degree',0.0174532925199433,ID'EPSG',8824],PARAMETER['Easting at false origin',0,LENGTHUNIT'metre',1,ID'EPSG',8826],PARAMETER['Northing at false origin',0,LENGTHUNIT'metre',1,ID'EPSG',8827]],CSCartesian,2,AXIS['easting',east,ORDER1,LENGTHUNIT'metre',1],AXIS['northing',north,ORDER2,LENGTHUNIT'metre',1],ID

Show the bounding box in the geographical coordinates

In [26]:
print(get_geographic_coordinate_system(gdf))

('WGS 84 geodetic', 4326)


In [60]:
from pyproj import Proj, transform

bbox_projected = gdf.total_bounds

# Define the projected coordinate system and the WGS84 coordinate system
projected_crs = gdf.crs
wgs84_crs = f"EPSG:{get_geographic_coordinate_system(gdf)[1]}"

# Create transformation functions
transformer = Transformer.from_crs(projected_crs, wgs84_crs)

# Transform the bounding box points to lat/lon
min_lon, min_lat = transformer.transform(bbox_projected[0], bbox_projected[1]) 
max_lon, max_lat = transformer.transform(bbox_projected[2], bbox_projected[3])

bbox_lonlat = (min_lon, min_lat, max_lon, max_lat)
print(bbox_lonlat)

(41.70049003694901, -111.78438452093438, 42.102360645589236, -111.51208495002092)
